You can integrate Snowflake notifications with the following external systems by using the webhooks that these systems provide:

* Slack
* Microsoft Teams
* PagerDuty

Snowflake does not send webhook notifications to external systems other than the ones listed above.

To send a notification to one of these systems:

1) Create the secret for the webhook URL for the external system.
2) Create the webhook notification integration for the external system.
3) Send the notification to the external system, using the webhook notification integration.

For this example, I am going to be connecting to Slack. There is some prework that needs to be done

Instructions for Slack can be found [here](https://api.slack.com/messaging/webhooks#create_a_webhook)

Instructions for Teams can be found [here](https://learn.microsoft.com/en-us/microsoftteams/platform/webhooks-and-connectors/how-to/add-incoming-webhook?tabs=newteams%2Cdotnet)

Instructions for PagerDuty can be found [here](https://support.pagerduty.com/main/docs/services-and-integrations)

I don't exactly know what will happen if multiple people use the same setup in Slack, but if you want to try, I will share my secret string.



In [ ]:
-- First, we'll create a secret object for this secret

USE DATABASE SNOW_WEBHOOKS;
USE SCHEMA DATA;

CREATE OR REPLACE SECRET my_slack_webhook_secret
  TYPE = GENERIC_STRING
  SECRET_STRING = '<REPLACE_THIS>';

To create a notification integration of the webhook type, use the CREATE NOTIFICATION INTEGRATION command.

When executing this command, set the following properties to set up the HTTP request that should be sent for the notification.

* Set TYPE to WEBHOOK.
* If you created a secret object for a secret to be included in the URL, HTTP request body, or header, set WEBHOOK_SECRET to the name of that secret object.
* Set WEBHOOK_URL to the URL for the webhook.
* If the webhook URL includes a secret and you created a secret object for this secret, replace the secret in the URL with SNOWFLAKE_WEBHOOK_SECRET.
* If the body of the message for the webhook needs to be in a specific format for this external system (for example, if all messages sent to this system need to use the same format), set WEBHOOK_BODY_TEMPLATE to a template for the message. In this template:
    * Use the SNOWFLAKE_WEBHOOK_SECRET placeholder where the secret should appear in the body of the message.
    * Use the SNOWFLAKE_WEBHOOK_MESSAGE placeholder where the notification message should appear.
* When you call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION and pass in a message, the stored procedure uses the template to construct the body of the webhook request. The stored procedure replaces the SNOWFLAKE_WEBHOOK_MESSAGE placeholder with the message that you pass in.
* If the HTTP request to the webhook must include specific HTTP headers, set WEBHOOK_HEADERS to the list of the header names and values.
* Use the SNOWFLAKE_WEBHOOK_SECRET placeholder where the secret should appear in the value of a header.

In [ ]:
-- We have our secret created already, it's named my_slack_webhook_secret

CREATE OR REPLACE NOTIFICATION INTEGRATION my_slack_webhook_int
  TYPE=WEBHOOK
  ENABLED=TRUE
  WEBHOOK_URL='https://hooks.slack.com/services/SNOWFLAKE_WEBHOOK_SECRET'
  WEBHOOK_SECRET=SNOW_WEBHOOKS.DATA.my_slack_webhook_secret
  WEBHOOK_BODY_TEMPLATE='{"text": "SNOWFLAKE_WEBHOOK_MESSAGE"}'
  WEBHOOK_HEADERS=('Content-Type'='application/json');

Placeholders like SNOWFLAKE_WEBHOOK_SECRET are used in notification integrations. When you create a notification integration, you can use placeholders to indicate where you want the content inserted into the request. For example, you can use the SNOWFLAKE_WEBHOOK_SECRET placeholder to insert the secret into the HTTP headers or body of the request.

Now we can send a notification to a webhook

To send a notification to a webhook:

1) Pass the SANITIZE_WEBHOOK_CONTENT function to remove any placeholders (like SNOWFLAKE_WEBHOOK_SECRET) from the message.
2) Call the SYSTEM$SEND_SNOWFLAKE_NOTIFICATION stored procedure, passing in the sanitized message and specifying the name of the webhook notification integration to use.

In [ ]:
-- For example, the following statement sends a JSON message to a Slack webhook, using the notification integration that you created earlier:

CALL SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
  SNOWFLAKE.NOTIFICATION.TEXT_PLAIN(
    SNOWFLAKE.NOTIFICATION.SANITIZE_WEBHOOK_CONTENT('Hi! This is my message')
  ),
  SNOWFLAKE.NOTIFICATION.INTEGRATION('my_slack_webhook_int')
);

In this example, the statement passes in a message in plain text (my message). When constructing the body of the webhook request from the template specified by the WEBHOOK_BODY_TEMPLATE property of the notification integration, SYSTEM$SEND_SNOWFLAKE_NOTIFICATION replaces the SNOWFLAKE_WEBHOOK_MESSAGE placeholder with the message that you pass in.

Lets make a quick alert that will utilize this webhook.

For this example, our alert is going to be very simple and we're going to do some manual work to get it to run.

First, lets create a table that hold some sample data.

In [ ]:
-- create our table
create or replace table TEST_ALERT_DATA(id INT, value varchar(50), isok boolean);

-- insert a row
insert into TEST_ALERT_DATA(id, value, isok) VALUES (1, 'Hi! Things are fine', true);

-- and lets verify
select * from TEST_ALERT_DATA;

Lets create a serverless alert that will let us know if things aren't ok.

1) Verify that you are using a role that has the privileges to create an alert.
2) Verify that you are using database and schema in which you plan to create the alert.
3) Execute the CREATE ALERT command to create the alert:

In [ ]:
CREATE OR REPLACE ALERT my_alert
  SCHEDULE = '1 minute'
  IF( EXISTS(
    SELECT value from TEST_ALERT_DATA where isok = FALSE))
  THEN
    BEGIN
      LET BAD_THINGS string := (select listagg(value, ', ') as bad_row_messages
        from TEST_ALERT_DATA where isok = FALSE);
      
       CALL SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
          SNOWFLAKE.NOTIFICATION.TEXT_PLAIN(
            SNOWFLAKE.NOTIFICATION.SANITIZE_WEBHOOK_CONTENT('Table contains the following bad things: ' || :BAD_THINGS || ',')
          ),
          SNOWFLAKE.NOTIFICATION.INTEGRATION('my_slack_webhook_int')
        );
    END;
;

-- Alerts are suspended by default, so we'll resume
ALTER ALERT my_alert RESUME;

-- and lets check the status of the alert
DESC ALERT my_alert;

We can view the run history easily

And if we don't do anything or make any changes, we can see details and that it run every minute.

In [ ]:
SELECT *
FROM
  TABLE(INFORMATION_SCHEMA.ALERT_HISTORY(
    SCHEDULED_TIME_RANGE_START
      =>dateadd('hour',-1,current_timestamp())))
ORDER BY SCHEDULED_TIME DESC;

In [ ]:
-- if we don't want to wait the minute, we can force the alert to execute
EXECUTE ALERT my_alert;

So, lets go aheand and check slack and we shouldn't see any messages.

Let's change that by inserting a bad row in our table.

In [ ]:
insert into TEST_ALERT_DATA(id, value, isok) VALUES (2, 'Oh no! Things are not fine', false);
insert into TEST_ALERT_DATA(id, value, isok) VALUES (3, 'Unfortunately, Things are getting worse', false);

and again, we can wait the minute or force things to run.

In [ ]:
-- if we don't want to wait the minute, we can force the alert to execute
EXECUTE ALERT my_alert;

Now, the way we have this setup, we are going to get a not ok message until the bad record is deleted. So every minute, we can expect a message that things aren't ok.

In [ ]:
-- Lets go ahead and suspend the alert so we don't get too many messages in slack.
ALTER ALERT my_alert SUSPEND;